In [8]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
import csv
import sys

def read_csv_file(csv_file_path):
    """
    CSV 파일을 읽어서 판다스 DataFrame으로 반환하는 함수
    
    Args:
        csv_file_path (str): CSV 파일 경로
    
    Returns:
        pandas.DataFrame: CSV 데이터가 담긴 DataFrame
    """
    try:
        # CSV 파일 읽기
        df = pd.read_csv(csv_file_path, encoding='utf-8')
        print(f"CSV 파일을 성공적으로 읽었습니다. 총 {len(df)} 개의 행이 있습니다.")
        
        # 컬럼 이름 출력
        print(f"컬럼 목록: {df.columns.tolist()}")
        
        return df
    
    except Exception as e:
        print(f"CSV 파일 읽기 중 오류 발생: {e}")
        return None

def create_database_connection(host_name, user_name, user_password, db_name=None):
    """
    MySQL 데이터베이스 연결을 생성하는 함수
    
    Args:
        host_name (str): MySQL 서버 호스트
        user_name (str): MySQL 사용자 이름
        user_password (str): MySQL 비밀번호
        db_name (str, optional): 데이터베이스 이름
    
    Returns:
        mysql.connector.connection.MySQLConnection: 데이터베이스 연결 객체
    """
    connection = None
    try:
        if db_name:
            connection = mysql.connector.connect(
                host=host_name,
                user=user_name,
                passwd=user_password,
                database=db_name
            )
            print(f"MySQL 데이터베이스 '{db_name}'에 성공적으로 연결했습니다.")
        else:
            connection = mysql.connector.connect(
                host=host_name,
                user=user_name,
                passwd=user_password
            )
            print("MySQL 서버에 성공적으로 연결했습니다.")
    except Error as e:
        print(f"MySQL 연결 중 오류 발생: {e}")
    
    return connection

def create_database(connection, db_name):
    """
    MySQL 데이터베이스를 생성하는 함수
    
    Args:
        connection (mysql.connector.connection.MySQLConnection): 데이터베이스 연결 객체
        db_name (str): 생성할 데이터베이스 이름
    
    Returns:
        bool: 성공 여부
    """
    cursor = connection.cursor()
    try:
        # 데이터베이스 존재 여부 확인
        cursor.execute(f"SHOW DATABASES LIKE '{db_name}'")
        result = cursor.fetchone()
        
        if result:
            print(f"데이터베이스 '{db_name}'가 이미 존재합니다.")
            return True
        
        # 데이터베이스 생성
        cursor.execute(f"CREATE DATABASE {db_name} CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci")
        print(f"데이터베이스 '{db_name}'를 성공적으로 생성했습니다.")
        return True
    
    except Error as e:
        print(f"데이터베이스 생성 중 오류 발생: {e}")
        return False
    
    finally:
        cursor.close()

def check_table_exists(connection, table_name):
    """
    테이블 존재 여부를 확인하는 함수
    
    Args:
        connection (mysql.connector.connection.MySQLConnection): 데이터베이스 연결 객체
        table_name (str): 확인할 테이블 이름
    
    Returns:
        bool: 테이블 존재 여부
    """
    cursor = connection.cursor()
    try:
        cursor.execute(f"SHOW TABLES LIKE '{table_name}'")
        result = cursor.fetchone()
        return result is not None
    except Error as e:
        print(f"테이블 확인 중 오류 발생: {e}")
        return False
    finally:
        cursor.close()

def map_and_insert_data(connection, table_name, df):
    """
    DataFrame의 데이터를 특정 테이블 구조에 맞게 매핑하여 MySQL 테이블에 삽입하는 함수
    - name -> point_name
    - address -> address
    - latitude -> latitude
    - longitude -> longitude
    - official -> 모두 true로 설정
    
    Args:
        connection (mysql.connector.connection.MySQLConnection): 데이터베이스 연결 객체
        table_name (str): 데이터를 삽입할 테이블 이름
        df (pandas.DataFrame): 삽입할 데이터가 담긴 DataFrame
    
    Returns:
        bool: 성공 여부
    """
    cursor = connection.cursor()
    try:
        # CSV에 필요한 컬럼이 있는지 확인
        required_cols = ['name', 'address', 'latitude', 'longitude']
        missing_cols = [col for col in required_cols if col not in df.columns]
        
        if missing_cols:
            print(f"CSV 파일에 필요한 컬럼이 누락되었습니다: {missing_cols}")
            return False
        
        # 테이블 구조 확인
        if not check_table_exists(connection, table_name):
            # 테이블이 없으면 생성
            create_query = f"""
            CREATE TABLE {table_name} (
                id INT AUTO_INCREMENT PRIMARY KEY,
                official BOOLEAN NOT NULL,
                address VARCHAR(255),
                latitude FLOAT,
                longitude FLOAT,
                point_name VARCHAR(255)
            ) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci
            """
            cursor.execute(create_query)
            print(f"테이블 '{table_name}'을 생성했습니다.")
        else:
            print(f"테이블 '{table_name}'이 이미 존재합니다.")
            # 기존 데이터 삭제 여부 확인
            confirm = input("기존 테이블의 데이터를 모두 삭제하고 새로운 데이터를 삽입하시겠습니까? (y/n): ")
            if confirm.lower() == 'y':
                cursor.execute(f"TRUNCATE TABLE {table_name}")
                print(f"테이블 '{table_name}'의 데이터를 모두 삭제했습니다.")
            else:
                print("기존 데이터를 유지한 채로, 새 데이터를 추가합니다.")
        
        # 데이터 삽입 쿼리 준비
        insert_query = f"""
        INSERT INTO {table_name} (official, address, latitude, longitude, point_name)
        VALUES (TRUE, %s, %s, %s, %s)
        """
        
        # 데이터 매핑 및 삽입
        values = []
        for _, row in df.iterrows():
            # name -> point_name, address -> address, latitude -> latitude, longitude -> longitude
            point_name = row['name']
            address = row['address'] if 'address' in row and pd.notna(row['address']) else None
            latitude = row['latitude'] if pd.notna(row['latitude']) else None
            longitude = row['longitude'] if pd.notna(row['longitude']) else None
            
            values.append((address, latitude, longitude, point_name))
        
        # 데이터 일괄 삽입
        cursor.executemany(insert_query, values)
        connection.commit()
        
        print(f"총 {cursor.rowcount} 개의 레코드가 '{table_name}' 테이블에 삽입되었습니다.")
        return True
    
    except Error as e:
        print(f"데이터 삽입 중 오류 발생: {e}")
        return False
    
    finally:
        cursor.close()

def csv_to_mysql_custom(csv_file_path, host, user, password, db_name, table_name):
    """
    CSV 파일의 데이터를 MySQL 데이터베이스에 맞춤형 매핑으로 저장하는 함수
    
    Args:
        csv_file_path (str): CSV 파일 경로
        host (str): MySQL 서버 호스트
        user (str): MySQL 사용자 이름
        password (str): MySQL 비밀번호
        db_name (str): 데이터베이스 이름
        table_name (str): 테이블 이름
    
    Returns:
        bool: 성공 여부
    """
    # CSV 파일 읽기
    df = read_csv_file(csv_file_path)
    if df is None:
        return False
    
    # MySQL 서버 연결
    connection = create_database_connection(host, user, password)
    if connection is None:
        return False
    
    # 데이터베이스 생성 또는 확인
    if not create_database(connection, db_name):
        connection.close()
        return False
    
    # 데이터베이스 선택
    connection.database = db_name
    
    # 데이터 매핑 및 삽입
    result = map_and_insert_data(connection, table_name, df)
    
    # 연결 종료
    connection.close()
    
    return result

def main():
    
    csv_file_path = "/home/chichi/S12P21C201/BE/fish_points/fishing_points_coastline.csv"
    
    # MySQL 연결 정보
    host = "j12c201.p.ssafy.io"
    user = "root"
    password = "ghkdcldns201"
    db_name = "TheWater"
    table_name = "fishing_point"
    
    # CSV 데이터를 MySQL에 저장
    if csv_to_mysql_custom(csv_file_path, host, user, password, db_name, table_name):
        print(f"CSV 데이터를 MySQL 데이터베이스 '{db_name}'의 '{table_name}' 테이블에 성공적으로 저장했습니다.")
    else:
        print("CSV 데이터를 MySQL에 저장하는 데 실패했습니다.")

if __name__ == "__main__":
    main()

CSV 파일을 성공적으로 읽었습니다. 총 1047 개의 행이 있습니다.
컬럼 목록: ['search_keyword', 'near_coastline', 'place_url', 'name', 'address', 'place_name', 'longitude', 'category', 'phone', 'latitude']
MySQL 서버에 성공적으로 연결했습니다.
데이터베이스 'TheWater'가 이미 존재합니다.
테이블 'fishing_point'이 이미 존재합니다.
기존 데이터를 유지한 채로, 새 데이터를 추가합니다.
총 1047 개의 레코드가 'fishing_point' 테이블에 삽입되었습니다.
CSV 데이터를 MySQL 데이터베이스 'TheWater'의 'fishing_point' 테이블에 성공적으로 저장했습니다.
